In [3]:
import numpy as np
import pandas as pd

In [4]:
for interp_type in ['linear', 'cubic', 'ncs', 'polynomial', 'fakenodes']:
    pop_df = pd.read_csv('../../data/population-by-' + interp_type + '.csv', index_col=0)
    states_list = list(pop_df.index)
    monthly_deaths = pd.read_csv('../../data/month-deaths.csv').drop("Unnamed: 0", 1)
    pop_df = pd.melt(pop_df)
    pop_df.index = states_list*13
    pop_df = pop_df.rename(columns={"variable": "Date", "value": "Population"})
    pop_df["Month"] = pd.DataFrame(pop_df["Date"].apply(lambda x: x.split("/")[0]).astype("int"))
    pop_df["Year"] = ("20" + pd.DataFrame(pop_df["Date"].apply(lambda x: x.split("/")[1]))).astype("int")
    pop_df["State"] = pop_df.index
    pop_and_deaths_df = pd.merge(monthly_deaths, pop_df.drop("Date", 1), how="inner", on=["Month","Year","State"])
    
    # removing total metric rows

    pop_and_deaths_df = pop_and_deaths_df[pop_and_deaths_df['Sex'] != "All Sexes"]
    pop_and_deaths_df = pop_and_deaths_df[pop_and_deaths_df['Age Group'] != "All Ages"]
    pop_and_deaths_df = pop_and_deaths_df[pop_and_deaths_df['Age Group'] != "0-17 years"]
    pop_and_deaths_df = pop_and_deaths_df[pop_and_deaths_df['State'] != "United States"]

    # removing states not covered in population interpolations

    pop_and_deaths_df = pop_and_deaths_df[pop_and_deaths_df['State'] != "Puerto Rico"]
    pop_and_deaths_df = pop_and_deaths_df[pop_and_deaths_df['State'] != "New York City"]
    
    # save merged data set

    pop_and_deaths_df.to_csv("../../data/monthly-deaths-with-population-by-" + interp_type + ".csv")
    
    # melting the mandate data set

    mandates_df = pd.read_csv("../../data/mandates_binary.csv")
    mandates_df.columns = ["State"] + list(pop_df["Date"].unique())+["Mandate Source"]
    mandates_df = mandates_df.set_index("State").drop("Mandate Source", 1)
    mandates_df = mandates_df.melt()
    mandates_df.index = states_list*13
    mandates_df = mandates_df.rename(columns={"variable": "Date", "value": "Mandate"})
    mandates_df["Month"] = pd.DataFrame(mandates_df["Date"].apply(lambda x: x.split("/")[0]).astype("int"))
    mandates_df["Year"] = ("20" + pd.DataFrame(mandates_df["Date"].apply(lambda x: x.split("/")[1]))).astype("int")
    mandates_df["State"] = mandates_df.index
    mandates_df = mandates_df.drop("Date", 1)
    
    # melting the non-binary mandate data set

    fluid_mandates_df = pd.read_csv("../../data/mandates_nonbinary.csv")
    fluid_mandates_df.columns = ["State"] + list(pop_df["Date"].unique())+["Mandate Source"]
    fluid_mandates_df = fluid_mandates_df.set_index("State").drop("Mandate Source", 1)
    fluid_mandates_df = fluid_mandates_df.melt()
    fluid_mandates_df.index = states_list*13
    fluid_mandates_df = fluid_mandates_df.rename(columns={"variable": "Date", "value": "Mandate"})
    fluid_mandates_df["Month"] = pd.DataFrame(fluid_mandates_df["Date"].apply(lambda x: x.split("/")[0]).astype("int"))
    fluid_mandates_df["Year"] = ("20" + pd.DataFrame(fluid_mandates_df["Date"].apply(lambda x: x.split("/")[1]))).astype("int")
    fluid_mandates_df["State"] = fluid_mandates_df.index
    fluid_mandates_df = fluid_mandates_df.drop("Date", 1)
    
    # save final merged datasets
    
    pop_deaths_mandates_df = pd.merge(pop_and_deaths_df, mandates_df, how="inner", on=["Month","Year","State"])
    pop_deaths_mandates_df.to_csv("../../data/" + interp_type + "-pop-deaths-and-binary-mandates.csv")
    pop_deaths_fluid_mandates_df = pd.merge(pop_and_deaths_df, fluid_mandates_df, how="inner", on=["Month","Year","State"])
    pop_deaths_fluid_mandates_df.to_csv("../../data/" + interp_type + "-pop-deaths-and-nonbinary-mandates.csv")